# Results and plotting

This tutorial covers how to inspect, export, and plot results from an STIsim simulation.

## Setup

Let's run a simple gonorrhea simulation to work with:

In [ ]:
import stisim as sti
import matplotlib.pyplot as plt

sim = sti.Sim(diseases='ng', n_agents=2000, start=2010, stop=2030)
sim.run(verbose=0)

## Browsing available results

After running a sim, all results are stored in `sim.results`. Each disease has its own results object. You can list all available keys with `sim.results.ng.keys()`, or access individual results directly. STIsim stores results at multiple levels of detail: overall, by sex, and by age group.

In [ ]:
# Access individual results
prev = sim.results.ng.prevalence
print(f'{prev.label}: min={prev.values.min():.3f}, mean={prev.values.mean():.3f}, max={prev.values.max():.3f}')

new_inf = sim.results.ng.new_infections
print(f'{new_inf.label}: min={new_inf.values.min():.0f}, mean={new_inf.values.mean():.1f}, max={new_inf.values.max():.0f}')

# There are many more -- including by sex and age group
print(f'\nTotal result keys: {len(sim.results.ng.keys())}')

Each result is an `ss.Result` object. The raw data is in `.values` (a NumPy array) and the time axis is in `.timevec`:

In [ ]:
print(f'Values (first 5): {prev.values[:5]}')
print(f'Timevec:          {prev.timevec[0]} to {prev.timevec[-1]} ({len(prev.values)} timesteps)')

## Automatic plotting

The simplest way to visualize results is `sim.plot()`. By default, it shows only the high-level results (not every age/sex subgroup):

In [ ]:
sim.plot()

### Plotting specific results

To plot specific results, pass the `key` argument. You can use a single key or a list:

In [ ]:
# Plot just prevalence and new infections
sim.plot(key=['ng.prevalence', 'ng.new_infections'])

You can also plot results that are hidden from the default view. For example, sex-stratified prevalence:

In [ ]:
sim.plot(key=['ng.prevalence_f', 'ng.prevalence_m'])

## Exporting results

### As a DataFrame

Export all results to a Pandas DataFrame with `sim.to_df()`:

In [ ]:
df = sim.to_df()
print(f'Shape: {df.shape}')
df.head()

You can also export a single result:

In [ ]:
prev_df = sim.results.ng.prevalence.to_df()
prev_df.head()

## Resampling to annual results

STIsim runs with monthly timesteps by default, but you'll often want annual results for reporting or comparison with data. Use the `resample()` method on any result:


In [ ]:
# Monthly new infections (raw)
monthly = sim.results.ng.new_infections
print(f'Monthly: {len(monthly.values)} timesteps')

# Resample to annual (sums monthly counts)
annual = monthly.resample('year')
print(f'Annual:  {len(annual)} years')
print(annual)

The `resample()` method automatically chooses the right aggregation: **sum** for counts (like `new_infections`) and **mean** for rates/proportions (like `prevalence`). You can also resample to a DataFrame:

In [ ]:
annual_df = sim.results.ng.new_infections.resample('year', output_form='dataframe')
annual_df

You can also resample the entire sim's results at once via `to_df()`:

In [ ]:
annual_all = sim.to_df(resample='year', use_years=True)
annual_all.head()

## Custom plots with Matplotlib

For publication-quality figures or custom layouts, use the result values directly with Matplotlib:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Left: prevalence by sex
axes[0].plot(sim.timevec, sim.results.ng.prevalence_f, label='Female')
axes[0].plot(sim.timevec, sim.results.ng.prevalence_m, label='Male')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Prevalence')
axes[0].set_title('NG prevalence by sex')
axes[0].legend()

# Right: annual new infections (resampled)
annual_inf = sim.results.ng.new_infections.resample('year', output_form='dataframe', use_years=True)
axes[1].bar(annual_inf.index, annual_inf['value'])
axes[1].set_xlabel('Year')
axes[1].set_ylabel('New infections')
axes[1].set_title('Annual NG infections')

plt.tight_layout()
fig

## Summary

| Task | Method |
|------|--------|
| List all result keys | `sim.results.ng.keys()` |
| Access a result's values | `sim.results.ng.prevalence.values` |
| Auto-plot high-level results | `sim.plot()` |
| Plot specific results | `sim.plot(key=['ng.prevalence', 'ng.new_infections'])` |
| Plot hidden results | `sim.plot(key='ng.prevalence_f')` |
| Export all to DataFrame | `sim.to_df()` |
| Export one result | `sim.results.ng.prevalence.to_df()` |
| Resample to annual | `result.resample('year')` |
| Annual DataFrame | `sim.to_df(resample='year', use_years=True)` |

## Exercises

1. Run a sim with both `'ng'` and `'ct'`. Use `sim.plot(key=...)` to compare the prevalence of the two diseases on the same figure.
2. Export the annual new infections for gonorrhea to a DataFrame and save it to a CSV file.
3. Make a Matplotlib figure that plots monthly symptomatic prevalence (`ng.symp_prevalence`) alongside total prevalence (`ng.prevalence`). What fraction of infections are symptomatic?